In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import __future__
from random import random
%matplotlib inline

In [3]:
img = cv2.imread('pictures/lena/lena.png').astype(np.float)

In [4]:
fix = lambda img: cv2.normalize(img, None, 0.0, 1.0, cv2.NORM_MINMAX)

In [5]:
def saltPepper(img, prob = 0.05):
    pepper = prob
    salt = 1-prob
    output = np.zeros(img.shape, np.float)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            rand = random()
            if rand < pepper:
                intensity = [0,0,0]
            elif rand > salt:
                intensity = [1,1,1]
            else:
                intensity = [img[i][j][0], img[i][j][1], img[i][j][2]]
            output[i][j] = intensity
    return output

In [6]:
contrastStretching = lambda img: (img-img.min())/(img.max()-img.min())*255.

In [7]:
def additiveGaussianNoise(img, mean = 0, var = 0.1):
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,img.shape)
    gauss = gauss.reshape(img.shape)
    return img + gauss

In [22]:
def jpegCompression(img):
    encode_param = [cv2.IMWRITE_JPEG_QUALITY, 90]
    _, encimg = cv2.imencode('.jpg', img, encode_param)
    return cv2.imdecode(encimg, 0).astype(np.float)

In [9]:
blurring = lambda img, width, height : cv2.blur(img, (width, height))

In [10]:
def speckleNoise(img, mean = 0, var = 0.1):
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,img.shape)
    gauss = gauss.reshape(img.shape)
    return img + img*gauss

In [25]:
cv2.imshow("Original", fix(img))
cv2.imshow("Contrast Stretching", fix(contrastStretching(img)))
cv2.imshow("Blurring", fix(blurring(img, 15, 15)))
cv2.imshow("Additive Gaussian Noise", fix(additiveGaussianNoise(img, var=100)))
cv2.imshow("Salt and Pepper", fix(saltPepper(img)))
cv2.imshow("Salt and Pepper", fix(jpegCompression(img)))
cv2.waitKey(0)
cv2.destroyAllWindows()